In [5]:
import pandas as pd
import numpy as np
import os
os.chdir("../")

In [2]:
no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

In [7]:
data = no_precambr_data
dataPrekambr = precambr_data
isPrekambr = True
isHard = True
startColIdx = 1
endColIdx = 3
multiplyRowsIdx = 2
colName1 = "Eon"
colName2 = "System"
questionPattern = "Prekambr - wybierz system ery"

if isPrekambr:
    df = dataPrekambr.iloc[:, [startColIdx,endColIdx]]
else:
    df = data.iloc[:, [startColIdx,endColIdx]]

df = df.loc[df.index.repeat(multiplyRowsIdx)]

random_answers = []
questions = []

if isHard:
    scope = np.concatenate((data["PIETRO"].unique(), data["ODDZIAL"].unique(), data["SYSTEM"].unique(), data["ERA"].unique(), 
                            dataPrekambr["Jednostka nieformalna"].unique(), dataPrekambr["Eon"].unique(), 
                            dataPrekambr["Era"].unique(), dataPrekambr["System"].unique()))
else:
    if isPrekambr:
        scope = dataPrekambr[colName2].unique()
    else:
        scope = data[colName2].unique()

for i in df[colName1]:
    
    if isPrekambr:
        temp_correct = dataPrekambr[dataPrekambr[colName1]==i][colName2].array
    else:
        temp_correct = data[data[colName1]==i][colName2].array

    indexes = np.nonzero(np.in1d(scope, temp_correct))[0]
    temp_scope = np.delete(scope, indexes)
    
    if isHard:
        temp = np.concatenate((np.random.choice(temp_scope,4), np.random.choice(temp_correct,1)))
    else:
        temp = np.concatenate((np.random.choice(temp_scope,3), np.random.choice(temp_correct,1)))

    np.random.shuffle(temp)

    if not isHard:
        temp = np.append(temp, np.nan)

    random_answers.append(temp)
    questions.append(f"{questionPattern} {i}")

p1 = pd.DataFrame(questions, columns=["question"])
p2 = pd.DataFrame(random_answers, columns=["A", "B", "C", "D", "E"]).drop_duplicates()
p3 = pd.concat([p1, p2], axis=1)

all_correct_answers = []

for index, row in p3.iterrows():
    temp_val1 = row["question"].split(" ")[-1]

    if temp_val1.isdigit() or temp_val1 in ("górna", "dolna", "środkowa", "górny", "środkowy", "dolny"):
        temp_val1 = row["question"].split(" ")[-2:]
        temp_val1 = " ".join(temp_val1)

    temp_abcde = row[["A", "B", "C", "D", "E"]].tolist()

    if isPrekambr:
        correct_answers = dataPrekambr[dataPrekambr[colName1]==temp_val1][colName2].tolist()
    else:
        correct_answers = data[data[colName1]==temp_val1][colName2].tolist()

    for answer in temp_abcde:
        if answer in correct_answers:
            all_correct_answers.append(answer)
            break

p4 = pd.concat([p3, pd.DataFrame(all_correct_answers, columns=["correct_answer"])], axis=1)